In [10]:
import salabim_plus as sim_plus
import salabim as sim
import multiplexer
import OPTX
import BFPD
import splitter
#import horn_antenna
import antenna_kit
import cover
import section_A
import section_B
import section_C
import section_D
#import section_E
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 1000

with open('newestoutput_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
    env = sim_plus.Environment(trace=out)
    
    # step 2: create the machines in the simulation environment
    # do there need to be things like assembly benches here?
    VSWR_CTI          =sim_plus.Machine(var_name='VSWR_CTI',        env=env)
    VSWR_CTI_DBG      =sim_plus.Machine(var_name='VSWR_CTI_DBG',     env=env)
    OPTX_CTI          =sim_plus.Machine(var_name='OPTX_CTI',         env=env)
    BFPD_CTI          =sim_plus.Machine(var_name='BFPD_CTI',         env=env)
    BFPD_CTI_DBG      =sim_plus.Machine(var_name='BFPD_CTI_DBG',     env=env)
#    VSWR_CTI2         =sim_plus.Machine(var_name='VSWR_CTI2',        env=env)
    # maybe make VSWR_CTI1 and VSWR_CTI2 into a machine group that 
    # goes by the name of VSWR_CTI
#    MOD_FEEDASSY_PAT  =sim_plus.Machine(var_name='MOD_FEEDASSY_PAT', env=env)
    static_CTI        =sim_plus.Machine(var_name='STATIC_CTI',       env=env)
    static_CTI_DBG    =sim_plus.Machine(var_name='STATIC_CTI_DBG',   env=env)
#    COND_EST          =sim_plus.Machine(var_name='COND_EST',         env=env)
#    RF_Fit_Test       =sim_plus.Machine(var_name='RF_FIT_TEST',      env=env)
#    RF_CFG            =sim_plus.Machine(var_name='RF_CFG',           env=env)
 
    assembly_bench_1  = sim_plus.Machine(var_name='assembly_bench_1', env=env)
    #how many assembly benches should we create?
    
    quality_bench_1   = sim_plus.Machine(var_name='quality_bench_1',  env=env)
    #how many quality benches should we create?
    
    forklift_1        = sim_plus.Machine(var_name='forklift_1', env=env)
    #make as many forklifts are there are production controllers. Make sure to add
    #them to the machine group below

    # step 3: make machine grouping for common processes
    assembly_bench = sim_plus.MachineGroup(var_name='assembly_bench', env=env,
                                           machines=[assembly_bench_1])
    quality_bench  = sim_plus.MachineGroup(var_name='quality_bench', env=env,
                                           machines=[quality_bench_1])
    forklift       = sim_plus.MachineGroup(var_name='forklift', env=env, 
                                           machines=[forklift_1])
    
    
    # step 4: create the workers in the simulation environment
    # how many workers actually are there??
    assembler                  =sim_plus.Worker(var_name='assembler',    env=env, capacity=1)
    technician                 =sim_plus.Worker(var_name='technician',   env=env, capacity=1)
    production_control         =sim_plus.Worker(var_name='production_control', env=env, capacity=1)
    qual_inspector             =sim_plus.Worker(var_name='qual_inspector',   env=env, capacity=1)
    
    
    # step 5: make the shift schedules for the workers
    
    shift_schedule = misc_tools.make_shifts(shift_duration=24*60, 
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    production_control_shift = sim_plus.ShiftController(worker=production_control,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    qual_inspector_shift = sim_plus.ShiftController(worker=qual_inspector,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    # step 6: point to the step creation function
    
    # if we pull data from the database, can we update the create_routing and pass in an argument of the 
    # arrays that we want them to pull from / reference?
    
    multiplexer_steps_func  = multiplexer.create_routing
    OPTX_steps_func         = OPTX.create_routing
    BFPD_steps_func         = BFPD.create_routing
    splitter_steps_func     = splitter.create_routing
#    horn_antenna_steps_func = horn_antenna.create_routing
    cover_steps_func        = cover.create_routing
    antenna_kit_steps_func  = antenna_kit.create_routing
    section_A_steps_func    = section_A.create_routing
    section_B_steps_func    = section_B.create_routing
    section_C_steps_func    = section_C.create_routing
    section_D_steps_func    = section_D.create_routing
#    section_E_steps_func    = section_E.create_routing


    # step 7: make all of the generators that determine when entites are made
    
    multiplexer_gen = sim_plus.EntityGenerator(var_name='multiplexer', 
                                         steps_func=multiplexer_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    antenna_kit_gen = sim_plus.EntityGenerator(var_name='antenna_kit', 
                                         steps_func=antenna_kit_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    OPTX_gen = sim_plus.EntityGenerator(var_name='OPTX', 
                                         steps_func=OPTX_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    BFPD_gen = sim_plus.EntityGenerator(var_name='BFPD', 
                                         steps_func=BFPD_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    splitter_gen = sim_plus.EntityGenerator(var_name='splitter', 
                                         steps_func=splitter_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    cover_gen = sim_plus.EntityGenerator(var_name='cover', 
                                         steps_func=cover_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
#    horn_antenna_gen = sim_plus.EntityGenerator(var_name='horn_antenna', 
#                                         steps_func=horn_antenna_steps_func, 
#                                         env=env,
#                                         arrival_type='ordered')
#    
    section_A_gen = sim_plus.EntityGenerator(var_name='section_A', 
                                          steps_func=section_A_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=4)
    section_B_gen = sim_plus.EntityGenerator(var_name='section_B', 
                                          steps_func=section_B_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=3)
    section_C_gen = sim_plus.EntityGenerator(var_name='section_C', 
                                          steps_func=section_C_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=2)
    section_D_gen = sim_plus.EntityGenerator(var_name='section_D', 
                                          steps_func=section_D_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=1)
#    section_E_gen = sim_plus.EntityGenerator(var_name='section_E', 
#                                          steps_func=section_E_steps_func,
#                                          env=env,
#                                          arrival_type='inv_based', 
#                                          cut_queue=True, 
#                                          inv_level=0)
    
    # step 8: create all of the inventory locations in the simulation environment
    
    # FLT4_storage         = sim_plus.Storage(var_name='FLT4', env=env)
    
    section_D_storage = sim_plus.Storage(var_name='section_D',env=env)
    
    
    multiplexer_kanban  = sim_plus.Kanban(var_name='multiplexer',env=env,
                                        kanban_attr=multiplexer.create_kanban_attrs(env._env_objs))
    antenna_kit_kanban  = sim_plus.Kanban(var_name='antenna_kit',env=env,
                                        kanban_attr=antenna_kit.create_kanban_attrs(env._env_objs))
    OPTX_kanban         = sim_plus.Kanban(var_name='OPTX',env=env,
                                        kanban_attr=OPTX.create_kanban_attrs(env._env_objs))
    BFPD_kanban         = sim_plus.Kanban(var_name='BFPD',env=env,
                                        kanban_attr=BFPD.create_kanban_attrs(env._env_objs))
    splitter_kanban     = sim_plus.Kanban(var_name='splitter',env=env,
                                        kanban_attr=splitter.create_kanban_attrs(env._env_objs))
#    horn_antenna_kanban = sim_plus.Kanban(var_name='horn_antenna',env=env,
#                                        kanban_attr=horn_antenna.create_kanban_attrs(env._env_objs))
    cover_kanban        = sim_plus.Kanban(var_name='cover',env=env,
                                        kanban_attr=cover.create_kanban_attrs(env._env_objs))
    section_A_kanban    = sim_plus.Kanban(var_name='section_A',env=env,
                                        kanban_attr=section_A.create_kanban_attrs(env._env_objs))
    section_B_kanban    = sim_plus.Kanban(var_name='section_B',env=env,
                                        kanban_attr=section_B.create_kanban_attrs(env._env_objs))
    section_C_kanban    = sim_plus.Kanban(var_name='section_C',env=env,
                                        kanban_attr=section_C.create_kanban_attrs(env._env_objs))
#    section_D_kanban     = sim_plus.Kanban(var_name='section_D',env=env,
#                                        kanban_attr=section_D.create_kanban_attrs(env._env_objs))
    
    
    
    # step 9: connect any entity boms and main_exits to its entity generator
    
    section_A_gen.bom = section_A.get_bom(env=env._env_objs)
    section_B_gen.bom = section_B.get_bom(env=env._env_objs)
    section_C_gen.bom = section_C.get_bom(env=env._env_objs)
    section_D_gen.bom = section_D.get_bom(env=env._env_objs)
#    section_E_gen.bom = section_E.get_bom(env=env._env_objs)
    
    multiplexer_gen.main_exit  = multiplexer_kanban
    OPTX_gen.main_exit         = OPTX_kanban
    BFPD_gen.main_exit         = BFPD_kanban
    splitter_gen.main_exit     = splitter_kanban
#    horn_antenna_gen.main_exit = horn_antenna_kanban
    cover_gen.main_exit        = cover_kanban
    antenna_kit_gen.main_exit  = antenna_kit_kanban
    section_A_gen.main_exit    = section_A_kanban
    section_B_gen.main_exit    = section_B_kanban
    section_C_gen.main_exit    = section_C_kanban
#    section_D_gen.main_exit    = section_D_kanban

    
    # step 10: activate the controlling components within the simulation environment
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    production_control_shift.activate(process='work')
    qual_inspector_shift.activate(process='work')
    
    # activate components
    multiplexer_gen.activate(process='arrive')
    OPTX_gen.activate(process='arrive')
    BFPD_gen.activate(process='arrive')
    splitter_gen.activate(process='arrive')
#    horn_antenna_gen.activate(process='arrive')
    cover_gen.activate(process='arrive')
    antenna_kit_gen.activate(process='arrive') 
    section_A_gen.activate(process='arrive')
    section_B_gen.activate(process='arrive')
    section_C_gen.activate(process='arrive')
    section_D_gen.activate(process='arrive')
#    section_E_gen.activate(process='arrive')
    
    # step 11: execute the simulation
    env.run(till=runtime)

Traceback (most recent call last):
  File "/Users/rachelosborne/opt/anaconda3/lib/python3.7/site-packages/salabim/salabim.py", line 4246, in step
    next(c._process)
  File "/Users/rachelosborne/opt/anaconda3/lib/python3.7/site-packages/salabim_plus/salabim_plus.py", line 986, in process
    self.order_gen.send_order(self.order_qty)
  File "/Users/rachelosborne/opt/anaconda3/lib/python3.7/site-packages/salabim_plus/salabim_plus.py", line 247, in send_order
    self.ordered_qty.set(qty)
AttributeError: 'EntityGenerator' object has no attribute 'ordered_qty'


SystemExit: 